In [10]:
function chooseMax(M)
    #Choose an arc that has the max range of uncertainty
    arc_split = findall(M .== maximum(M))[1]
    return arc_split
end

function chooseScore(M, split_Candidates)
    #Choose an arc that has the lowest Score from Tree Sensitivity 
    #Analysis when Max_Range_Unc/Min_Range_Unc <= 10
    
    Delta_Final_Pos = ones(Len).*-1
    Delta_Final_Neg = ones(Len).*-1

    M_max = maximum(M)
    temp = find(M .>0)
    M_min = M_max
    for e in temp
        if M[e] <= M_min
            M_min = M[e]
        end
    end
    if M_max/M_min > 10
#                             println("Pick Max")
        split_Candidates[k] = findall(M .== maximum(M))[1]
    end
    if M_max/M_min <= 10
#                             println("SA")
        #FIND TREE, PRED
        y, gx, SP, T, pred, label, path = gx_bound(c_L, c_U, c, c_g, x_now, edge)
        g[k] = gx
        basic, basic_cost, i_basic, nonbasic, nonbasic_cost, i_nonbasic = classify_edges_costs(T,c_g)
        orderedArcList = getSortedBasicArcList(pred, basic, i_basic)

        label_temp = copy(label)
        c_temp = copy(c_g)

        for e in i_nonbasic
            u = edge[e,1]
            v = edge[e,2]
            Delta_Final_Neg[e] = label_temp[u] - label_temp[v] + c_temp[e]
        end

        for i = 1 : length(orderedArcList)
            e = orderedArcList[i]
            myNode = edge[e,2]
            if y[e] == 1 #if arc is on the shortest path y
                Delta_Final_Pos[e] = find_basic_Delta(myNode, Delta_Final_Pos, Delta_Final_Neg, "POS", basic, i_basic, nonbasic, i_nonbasic) 
                #Do not have to find Delta_Final_Neg : all set to (-1)
            else
                Delta_Final_Neg[e] = find_basic_Delta(myNode, Delta_Final_Pos, Delta_Final_Neg, "NEG", basic, i_basic, nonbasic, i_nonbasic)
            end
        end          

        arcs_neg = find(Delta_Final_Neg .>= 0)
        arcs_pos = find(Delta_Final_Pos .>= 0)
        found_split_arc = false
        arcs_delta = []
        myMax = maximum(Delta_Final_Neg)

        arcs = find(M.>0)                        
        if myMax < maximum(Delta_Final_Pos)
            myMax = maximum(Delta_Final_Pos)
        end

        myScore = myMax/M_min
        for i in arcs
#                                 println("i = ", i)
            if Delta_Final_Pos[i] >= 0 && Delta_Final_Pos[i]/M[i] < myScore
                myScore = Delta_Final_Pos[i]/M[i]
                split_Candidates[k] = i
                found_split_arc = true
            end
            if Delta_Final_Neg[i] >= 0 && Delta_Final_Neg[i]/M[i] < myScore
                myScore = Delta_Final_Neg[i]/M[i]
                split_Candidates[k] = i
                found_split_arc = true
            end
        end
        if found_split_arc == false
            split_Candidates[k] = find(M .== maximum(M))[1]
        end
    end
    return split_Candidates[k]
end

function chooseWorst(M, yL, yW)
    #After solving (3) at cL, solve (3) at cW where cW is worst for
    #yL. Compare yL and yW to obtain a set of candidate arcs. Choose
    #arc with largest uncertainty range.
    candidates = findall(yL.!=yW)
    arc_split = findall(M .== maximum(M[candidates]))[1]
    return arc_split
end

chooseWorst (generic function with 1 method)

1

In [28]:
yL_k = zeros(40)# [1 0 1]
yL_k[1] = 1
yL_k[5] = 1
yL_k[10] = 1
yL_k[20] = 1
yL_k[15] = 1
yL_k[27] = 1

yW_k = zeros(40)# [1 0 1]
yW_k[2] = 1
yW_k[5] = 1
yW_k[12] = 1
yW_k[20] = 1
yW_k[17] = 1
yW_k[35] = 1

c_L = collect(1:40)
x_now = zeros(40)
x_now[1] = 1
x_now[2] = 1
d = ones(40)
cg_k = c_L + d.*x_now
M = collect(51:90)
c_W = cg_k + yL_k.*M
println(c_W[findall(c_W.!=c_L)])

[53.0, 3.0, 60.0, 70.0, 80.0, 90.0, 104.0]


In [23]:
 yL_k.*M

40-element Array{Float64,1}:
 51.0
  0.0
  0.0
  0.0
 55.0
  0.0
  0.0
  0.0
  0.0
 60.0
  0.0
  0.0
  0.0
  ⋮  
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0

In [ ]:
using TimerOutputs

In [35]:
start = time()
for i = 1:10000000
    cW = copy(cg_k)
    for arc in findall(yL_k.==1)
        cW[arc] +=  M[arc]
    end
end 
total_time = time() - start

17.30299997329712

In [33]:
start = time()
for i = 1:10000000
    cW = cg_k + yL_k.*M
end 
total_time = time() - start

4.833000183105469

In [43]:
start = time()
for i = 1:10000000
    temp = findall(yL_k.!=yW_k)
    ind_ = findall(temp.==maximum(temp))[1]
end 
total_time = time() - start

21.128000020980835

In [45]:
start = time()
for i = 1:10000000
    M1 = M.*yL_k
    M2 = M.*yW_k
    temp = 0
    ind_ = 0
    for j = 1:length(M)
        if temp < M1[j]
            temp = M1[j]
            ind = j
        elseif temp < M2[j]
            temp = M2[j]
            ind_ = j
        end
    end
#     myMax = maximum(M[yL_k.!=yW_k])
#     ind_ = findall(M.==myMax)[1]
#     findall(yL_k.!=yW_k)
end 
total_time = time() - start

48.206000089645386

In [40]:
a = [1 0 0]
b = [0 1 0]
M1 = [4 5 6]
M1[a.!=b]

2-element Array{Int64,1}:
 4
 5